<b>[Author]</b> Nicolas Bosc
<br><b>[Year]</b> 2022

# Data extraction from ChEMBL
This notebook shows how to extract bioactivity data from the ChEMBL database to get them in a model training-friendly format. <br>
It makes use of the Python client library. <u>Therefore, it does not require a local installation of ChEMBL to run.</u>

To work, it only needs a protein name (by default COX-2) or alternatively its ChEMBL identifier. If data are found it writes a csv file with the relevant data

<b>Note</b>: there are several ways to achieve the same result and this notebook only show one possibility. Further documentation and examples are available [here](https://chembl.gitbook.io/chembl-interface-documentation/web-services/chembl-data-web-services). For remarks and comments please contact Nicolas Bosc <nbosc@ebi.ac.uk>

In [1]:
# Tested with Python 3.7
# You can install the required packages if they are not already installed. Just uncomment the next three lines.
# import sys
# !conda install --yes --prefix {sys.prefix} pandas ipywidgets
# !{sys.executable} -m pip install chembl-webresource-client rdkit-pypi

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client
import ipywidgets as w
#from IPython.display import display, Javascript
from rdkit.Chem import PandasTools

def find_target_in_chembl(widget_args, species='Homo sapiens'):
    protein_name = widget_args.kwargs['protein']
    # create a target query
    target = new_client.target
    # assume this is a 'single protein' present in the user-defined species
    response = target.filter(target_synonym__icontains=protein_name, organism=species, target_type='SINGLE PROTEIN')
    df_res = pd.DataFrame(response)
    return(df_res[['pref_name','target_chembl_id','organism']])

def find_activity_data(target_selection, argument):
    '''
    input: target selected from the list or chembl_id entered in the field
    Look for all the bioactivity in ChEMBL for this target. Restricted to data with pchembl values (-log(IC50, Ki, Kd, EC50...))
    Apply several sanity filters to keep only high confidence data
    ouput: dataframe with all the activities that pass the check
    '''
    
    if argument.kwargs['chembl_id'] == '':
        target_id = target_selection.value
    else:
        target_id = argument.kwargs['chembl_id']
    
    # Create an activity query
    activities = new_client.activity

    # Select only activities with a pchembl_value (-log(IC50, Ki, Kd, EC50...).
    # We also use the chembl flags to remove the duplicates and the records where there is a validity comment
    response = activities.filter(target_chembl_id=target_id, pchembl_value__isnull=False,\
                                 potential_duplicate=False, data_validity_comment__isnull=True )

    # create a dataframe with the activity data
    df_activities = pd.DataFrame(response)

    # create an assay query
    assays = new_client.assay
    # select assays.
    response = assays.filter(assay_chembl_id__in=list(df_activities.assay_chembl_id.unique()))

    # create a dataframe with the assay data
    df_assays = pd.DataFrame(response)

    # keep only the assays where the link between the protein target and the assay is direct
    df_assays = df_assays[df_assays.confidence_score==9]

    df_activities = df_activities[df_activities.assay_chembl_id.isin(df_assays.assay_chembl_id)]
    df_activities = df_activities.astype({'pchembl_value':float, 'standard_value':float})

    # keep only the columns you need
    df_res = df_activities[['assay_chembl_id','assay_description','parent_molecule_chembl_id','molecule_chembl_id','canonical_smiles','pchembl_value',\
                   'standard_type','standard_relation','standard_value','standard_units','target_pref_name',\
                   'target_chembl_id', 'target_organism']]
    print(f'{df_res.shape[0]} datapoint were found')
    return(df_res)

def remove_duplicates(df, do):
    '''
    if keep==True, remove duplicated data points
    based on all the values availables for a given compound on a given target,
    if the standard deviation < 1, then calculate the median value
    else don't keep the values
    '''
    if do:
        df_res = pd.DataFrame.copy(df)
        for cpd_id in df['parent_molecule_chembl_id'].unique():
            std = df[df.parent_molecule_chembl_id==cpd_id]['pchembl_value'].std()
            if std > 1:
                df_res = df_res[df_res.parent_molecule_chembl_id!=cpd_id]
        pchembl_median = df_res.groupby('parent_molecule_chembl_id')['pchembl_value'].median().reset_index()['pchembl_value']
        df_res  = df_res.drop_duplicates(['parent_molecule_chembl_id'])
        df_res = df_res.assign(pchembl_median=pchembl_median.values).drop('pchembl_value',axis=1)
        return(df_res)
    else:
        return(df)

def assay_summary(df):
    aff = df[(df.assay_description.str.contains('affinity', case=False))]['assay_chembl_id'].to_list()
    disp = df[(df.assay_description.str.contains('displacement', case=False))]['assay_chembl_id'].to_list()
    inhi = df[(df.assay_description.str.contains('inhibition', case=False))]['assay_chembl_id'].to_list()
    return(pd.DataFrame({'assay_type':['affinity','displacement','inhibition'], 'data':[len(aff),len(disp),len(inhi)]}))

def write_sdf(data, smiles_column, id_column, output_name):
    PandasTools.AddMoleculeColumnToFrame(data, smiles_column)

    # Uncomment the two lines below if a NoneType error appears when executing WriteSDF
    #     no_mol = data[data['ROMol'].isna()]
    #     data.drop(no_mol.index, axis=0, inplace=True)

    # add H
    # data.loc[:,'ROMol'] = [Chem.AddHs(x) for x in data.loc[:,'ROMol'].values.tolist()]

    PandasTools.WriteSDF(data, output_name, molColName='ROMol', properties=list(data.columns), idName=id_column)

##  Download activities for a given protein target

### Step 1: Looking for a target without have its ChEMBL id (ChEMBL id is known, go to [step 2](#Step-2))

In [5]:
def f(protein):
    return protein
target_argument = w.interactive(f, protein='')
target_argument

interactive(children=(Text(value='', description='protein'), Output()), _dom_classes=('widget-interact',))

In [26]:
targets = find_target_in_chembl(target_argument, species='Homo sapiens')
targets

,pref_name,target_chembl_id,organism
0,Serotonin 2b (5-HT2b) receptor,CHEMBL1833,Homo sapiens


In [27]:
target_selection = w.Select(
    options=[val for val in zip(targets['pref_name'],targets['target_chembl_id'])],
    description='Targets',
    disabled=False
)
print('Select the protein of interest from the list below')
target_selection

Select the protein of interest from the list below


Select(description='Targets', options=(('Serotonin 2b (5-HT2b) receptor', 'CHEMBL1833'),), value='CHEMBL1833')

### Step 2:  Looking for a target by ChEMBL id (written CHEMBL1234)
### Ignore if you followed step 1

In [8]:
def f(chembl_id):
    return chembl_id
chemblid_argument = w.interactive(f, chembl_id='')
chemblid_argument

interactive(children=(Text(value='', description='chembl_id'), Output()), _dom_classes=('widget-interact',))

### Step 3: Retrieve the activity data in ChEMBL 

In [32]:
df_activities = find_activity_data(target_selection, chemblid_argument)

HttpApplicationError: Error for url https://www.ebi.ac.uk/chembl/api/data/activity.json, server response: <!doctype html>
<!-- paulirish.com/2008/conditional-stylesheets-vs-css-hacks-answer-neither/ -->
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en"> <![endif]-->
<!-- Consider adding an manifest.appcache: h5bp.com/d/Offline -->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en"> <!--<![endif]-->
<head>
    <meta charset="utf-8">

    <!-- Use the .htaccess and remove these lines to avoid edge case issues.
 More info: h5bp.com/b/378 -->
    <!-- <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"> --> <!-- Not yet implemented -->

    <title>Server error &lt; EMBL-EBI</title>
    <meta name="description" content="EMBL-EBI"><!-- Describe what this page is about -->
    <meta name="keywords" content="bioinfor

#### Data endpoints available 

In [10]:
df_endpoints = pd.DataFrame(df_activities.standard_type.value_counts()).rename({'standard_type':'data points'},axis=1)
df_endpoints

,data points
IC50,2401
Ki,16
Kd,7
EC50,1


### Step 4: Based on the data retrieved, select the activity endpoint to use in the model.
Multiple values can be selected with <kbd>shift</kbd> and/or <kbd>ctrl</kbd> (or <kbd>command</kbd>) pressed and mouse clicks or arrow keys.

In [11]:
endpoint_selection = w.SelectMultiple(
    options=df_endpoints.index,
    description='Endpoints',
    disabled=False
)
endpoint_selection

SelectMultiple(description='Endpoints', options=('IC50', 'Ki', 'Kd', 'EC50'), value=())

In [12]:
df_activities = df_activities[df_activities.standard_type.isin(endpoint_selection.value)]

### Step 5: Should the duplicted value be removed? 

In [13]:
duplicate_selection = w.Select(
    options=[('Yes',True),('No',False)],
#     value=['CHEMBL1862'],
    description='remove duplicates?',
    disabled=False,
    style= {'description_width': 'initial'}
)
duplicate_selection

Select(description='remove duplicates?', options=(('Yes', True), ('No', False)), style=DescriptionStyle(descri…

In [14]:
df_activities = remove_duplicates(df_activities, duplicate_selection.value)

### Step 6: Select the type of assays
By defaults, assays are divided in 3 categories depending on whether their description contain certain words:
- affinity assay
- displacement assay
- inhibition assay

In [15]:
df_assays = assay_summary(df_activities)
df_assays

,assay_type,data
0,affinity,38
1,displacement,0
2,inhibition,2269


In [16]:
assay_selection = w.SelectMultiple(
    options=df_assays.assay_type,
    description='Assay types',
    disabled=False
)
assay_selection

SelectMultiple(description='Assay types', options=('affinity', 'displacement', 'inhibition'), value=())

In [17]:
df_activities = df_activities[df_activities.assay_description.str.contains('|'.join(assay_selection.value), case=False)]

In [18]:
df_activities.shape

(38, 13)

### Step 6: Export data for Flame 

Adapted by Eric Marc and Manuel Pastor (UPF), 2021
<br>Remove all the lines of this tables containing compounds without structure (the "canonical_smiles" is a na) and Write the SDFile

In [24]:
df_activities.drop(df_activities[df_activities['canonical_smiles'].isna()].index, axis=0, inplace=True)
write_sdf(df_activities, 'canonical_smiles', 'molecule_chembl_id', 'chembl_data.sdf')